# Estudo Biblioteca Keras explorando o uso da GPU / CPU
- O objetivo deste notebook é o estudo da Biblioteca Keras.
- Keras é uma biblioteca para rede neural de alto-nível escrita em Python. Ela roda como frontend em TensorFlow ou Theano. 
- Uma das grandes vantagens de utilizar essa biblioteca é que podemos substituir uma rede neural por outra de maneira muito fácil, pois ela foi criada com intuito de tornar essas experimentações mais fáceis, rápidas e eficientes. Com o Keras você pode abstrair tudo que acontece em background e focar no seu problema.
- Mais detalhes na página do projeto disponível neste <a href="https://keras.io/">link</a>.

<h3> Onde usar o Keras? </h3>

- Prototipagem rápida e fácil (total modularidade, minimalismo e extensibilidade).
- Suporte a redes convolucionais e recorrentes, incluindo combinação de ambas.
- Suporte a esquemas de conectividade arbitrária (incluindo treino de N para N).
- Rodar na CPU ou GPU.

<h3> DataSet </h3>
- Para este estudo vamos utilizar o data set 'census.csv'.</br>
- Este dataset contém 15 atributos e 32.561 instâncias (linhas). </br>

<h3> Ambiente dos Testes de CPU e GPU </h3>
- Estes testes foram realizados no notebook "Dell G7 15 7588" </br>
- Memória: 16GB (2x8GB) 2666MHz DDR4 Memory </br>
- Processador: CPU(8th Generation Intel(R) Core(TM) i7-8750H Processor (6-Core, 9MB Cache, up to 4.1GHz w/ Turbo Boost)) </br>
- Placa de Vídeo: GPU (Nvidia GeForce GTX 1060) </br>
- Linux Ubuntu 18.04 (kernel 5.0.0-25-generic)</br>
- Driver Version: 430.26</br>
- CUDA Version: 10.2 </br>
- Keras Version: '2.2.4'</br>
- TensorFlow Version: '1.14.0'</br>
- NVIDIA-SMI 430.26 </br>             

In [1]:
# Importando as bibliotecas iniciais
import pandas as pd
import numpy as np
from datetime import datetime
import os
import math
import tensorflow as tf

In [2]:
# Importa o dataset
base = pd.read_csv('./census.csv')

In [3]:
# Verificando o shape do dataset
base.shape

(32561, 15)

In [4]:
# Visualiza as primeira linhas
base.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [5]:
# Verificando os atributos núméricos.
base.describe()

,age,final-weight,education-num,capital-gain,capital-loos,hour-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


# Pré-processamento - Tratamento dos dados

In [6]:
# Seleciona os atributos previsores
previsores = base.iloc[:, 0:14].values
previsores

array([[39, ' State-gov', 77516, ..., 0, 40, ' United-States'],
       [50, ' Self-emp-not-inc', 83311, ..., 0, 13, ' United-States'],
       [38, ' Private', 215646, ..., 0, 40, ' United-States'],
       ...,
       [58, ' Private', 151910, ..., 0, 40, ' United-States'],
       [22, ' Private', 201490, ..., 0, 20, ' United-States'],
       [52, ' Self-emp-inc', 287927, ..., 0, 40, ' United-States']],
      dtype=object)

In [7]:
# Seleciona a classe alvo (target)
classe = base.iloc[:,14]
classe.head()

0     <=50K
1     <=50K
2     <=50K
3     <=50K
4     <=50K
Name: income, dtype: object

In [8]:
# Realiza a codificação dos atributos categóricos. Transforma as categorias em códigos.
from sklearn.preprocessing import LabelEncoder

label_encoder_previsores = LabelEncoder()
previsores[:,1] = label_encoder_previsores.fit_transform(previsores[:,1])
previsores[:,3] = label_encoder_previsores.fit_transform(previsores[:,3])
previsores[:,5] = label_encoder_previsores.fit_transform(previsores[:,5])
previsores[:,6] = label_encoder_previsores.fit_transform(previsores[:,6])
previsores[:,7] = label_encoder_previsores.fit_transform(previsores[:,7])
previsores[:,8] = label_encoder_previsores.fit_transform(previsores[:,8])
previsores[:,9] = label_encoder_previsores.fit_transform(previsores[:,9])
previsores[:,13] = label_encoder_previsores.fit_transform(previsores[:,13])
previsores

array([[39, 7, 77516, ..., 0, 40, 39],
       [50, 6, 83311, ..., 0, 13, 39],
       [38, 4, 215646, ..., 0, 40, 39],
       ...,
       [58, 4, 151910, ..., 0, 40, 39],
       [22, 4, 201490, ..., 0, 20, 39],
       [52, 5, 287927, ..., 0, 40, 39]], dtype=object)

In [9]:
# Realiza a codificação da classe (target). Transforma as categorias em códigos.
labelencoder_classe = LabelEncoder()
classe = labelencoder_classe.fit_transform(classe)

In [10]:
# Realiza a binarização dos valores inteiros. Técnica chamada: Dummy variables.
# Isto é importante para que o algoritmo não considere um valor maior mais importante que um valor menor. 
# Não tivemos ganhos com ele, por isso vamos deixar dasabilitado.

#from sklearn.preprocessing import OneHotEncoder

#oneHotEnconder = OneHotEncoder(categories='auto')
#previsores = oneHotEnconder.fit_transform(previsores).toarray()
#previsores

In [11]:
# StandardScaler para colocar todos os valores na mesma escala (escalonamento).
# Teste do escalonamente de todos atributos, tivemos melhoria no modelo. 
# Tivemos praticamente o mesma Acurácia da padronização parcial (abaixo). Por isso, vamos deixar ele dasabilitado.

#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#previsores = scaler.fit_transform(previsores)

In [12]:
# StandardScaler para colocar todos os valores na mesma escala (escalonamento).
# Teste do escalonamente somente dos atributos inteiros, também tivemos o mesmo resultado da padronização total. Tivemos praticamente o mesma Acurácia com o escalonamento total. 
# Vamos deixar ele habilitado.

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

previsores[:,0] =  scaler.fit_transform(previsores[:,0].reshape(-1,1)).ravel()
previsores[:,2] =  scaler.fit_transform(previsores[:,2].reshape(-1,1)).ravel()
previsores[:,4] =  scaler.fit_transform(previsores[:,4].reshape(-1,1)).ravel()
previsores[:,10] =  scaler.fit_transform(previsores[:,10].reshape(-1,1)).ravel()
previsores[:,11] =  scaler.fit_transform(previsores[:,11].reshape(-1,1)).ravel()
previsores[:,12] =  scaler.fit_transform(previsores[:,12].reshape(-1,1)).ravel()

In [13]:
# Separa os dados pata treinamento e teste
from sklearn.model_selection import train_test_split
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe , test_size = 0.25, random_state = 0)


In [14]:
# Desabilita a Execução por GPU.
# Força o uso da CPU.

forcar_Uso_CPU = True

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

if forcar_Uso_CPU:
    print("Forçando o uso da CPU.")
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
else:
    print("Forçando o uso da GPU.")
    os.environ["CUDA_VISIBLE_DEVICES"] = '0'

# Use o "nvidia-smi -l 2" no terminal para monitorar o uso da GPU (refresh a cada 2 segundos).
# https://homepages.uc.edu/~schreihf/uchenry/post/using-gpus/

Forçando o uso da CPU.


In [15]:
# Criando a função para formatar a exibição dos bytes
def convert_size(size_bytes):
   if size_bytes == 0:
       return "0B"
   size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
   i = int(math.floor(math.log(size_bytes, 1024)))
   p = math.pow(1024, i)
   s = round(size_bytes / p, 2)
   return "%s %s" % (s, size_name[i])

In [16]:
# Exibe a lista de recursos da sessão.
with tf.Session() as sess:
  devices = sess.list_devices()

print('Quantidade de recursos da sessão: {}'.format(len(devices)))
for dev in devices:
    print("\nTipo: {} | Nome: {} | Limite de memória: {}".format( dev.device_type, dev.name.split(':')[-2], convert_size(dev.memory_limit_bytes) ))
    

Quantidade de recursos da sessão: 2

Tipo: CPU | Nome: CPU | Limite de memória: 256.0 MB

Tipo: XLA_CPU | Nome: XLA_CPU | Limite de memória: 16.0 GB


# Criando o Modelo de Previsão

In [17]:
# Cria o modelo de previsão 
# import keras

from keras.models import Sequential
from keras.layers import Dense

# Cria camada de entrada da rede
classificador = Sequential()

# Cria a 1 camada oculta da rede
classificador.add(Dense(units = 8, activation = 'relu', input_dim = 14))

# Cria a 2 camada oculta da rede
classificador.add(Dense(units = 8, activation = 'relu'))

# Cria a camada de saida da rede
classificador.add(Dense(units = 1, activation = 'sigmoid'))

# Aplica as configurações no modelo para o treinamento.
classificador.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


Using TensorFlow backend.
W0816 15:19:17.454331 140579071416128 deprecation_wrapper.py:119] From /home/charles/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0816 15:19:17.455577 140579071416128 deprecation_wrapper.py:119] From /home/charles/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0816 15:19:17.457187 140579071416128 deprecation_wrapper.py:119] From /home/charles/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0816 15:19:17.481917 140579071416128 deprecation_wrapper.py:119] From /home/charles/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.trai

In [18]:
#Define o tempo de inicio do Treinamento
inicio = datetime.now()

# Realiza o treinamento da rede.
classificador.fit(previsores_treinamento, classe_treinamento, batch_size = 10, epochs = 100)

#Define o tempo final do Treinamento
fim = datetime.now()   
tempo_modelo = (fim - inicio).total_seconds()


W0816 15:19:17.608872 140579071416128 deprecation_wrapper.py:119] From /home/charles/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/100
24420/24420 [==============================] - 2s 83us/step - loss: 0.4476 - acc: 0.7763
Epoch 2/100
24420/24420 [==============================] - 2s 74us/step - loss: 0.3682 - acc: 0.8290
Epoch 3/100
24420/24420 [==============================] - 2s 62us/step - loss: 0.3413 - acc: 0.8388
Epoch 4/100
24420/24420 [==============================] - 2s 65us/step - loss: 0.3360 - acc: 0.8428
Epoch 5/100
24420/24420 [==============================] - 2s 73us/step - loss: 0.3352 - acc: 0.8430
Epoch 6/100
24420/24420 [==============================] - 1s 60us/step - loss: 0.3340 - acc: 0.8441
Epoch 7/100
24420/24420 [==============================] - 2s 72us/step - loss: 0.3336 - acc: 0.8437
Epoch 8/100
24420/24420 [==============================] - 3s 112us/step - loss: 0.3322 - acc: 0.8433
Epoch 9/100
24420/24420 [==============================] - 3s 106us/step - loss: 0.3318 - acc: 0.8452
Epoch 10/100
24420/24420 [==============================] - 2s 82us/step - loss: 0.3316 -

In [19]:
print('\nTempo de Treinamento: %s segundos' % tempo_modelo )


Tempo de Treinamento: 169.659337 segundos


In [20]:
# Resultados:

# Tempo de Treinamento usando a GPU:  469.941199 segundos
# Tempo de Treinamento usando a CPU:  169.659337 segundos    

In [21]:
# Realizando o testes do modelo
classe_prevista_teste = classificador.predict(previsores_teste)
classe_prevista_teste

array([[0.01531097],
       [0.01893675],
       [0.29823634],
       ...,
       [0.2565147 ],
       [0.6970377 ],
       [0.03089911]], dtype=float32)

In [22]:
# Converte os resultados para conferencia
classe_prevista_teste = (classe_prevista_teste > 0.5)
classe_prevista_teste

array([[False],
       [False],
       [False],
       ...,
       [False],
       [ True],
       [False]])

# Verificando a performance do Modelo

In [23]:
# Analisando a performance do Modelo
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
precisao_modelo = accuracy_score(classe_teste, classe_prevista_teste)
print('Acurácia do Modelo: %.2f%% ' % (precisao_modelo*100))


Acurácia do Modelo: 84.67% 


In [24]:
# Relatório de Classificação
from sklearn.metrics import classification_report

#precision -  daqueles que classifiquei como corretos, quantos efetivamente estavam corretos? 
#recall    -  quando realmente é da classe XPTO, o quão frequente você classifica como XPTO?
#f1-score  -  balanço entre a precisão e o recall. Combina precisão e recall em um número único que indique a qualidade geral do seu modelo (quanto maior melhor o modelo).
#support   -  número de ocorrência de cada classe.

relatorio_classificacao = classification_report(classe_teste, classe_prevista_teste ,digits=4)

# Imprimindo o relatório
print('\tRelatório de Classificação do modelo')
print(relatorio_classificacao)

	Relatório de Classificação do modelo
              precision    recall  f1-score   support

           0     0.8662    0.9430    0.9030      6159
           1     0.7556    0.5474    0.6349      1982

    accuracy                         0.8467      8141
   macro avg     0.8109    0.7452    0.7689      8141
weighted avg     0.8393    0.8467    0.8377      8141



In [25]:
# Visualizando a Matriz de confusão (confusion matrix)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(classe_teste, classe_prevista_teste) 

df_cm = pd.DataFrame(cm)

print('\n')
print('Obs.: Nas linhas temos as Classes reais e nas colunas as Classes previstas.\n')
print('Matriz de Confusão')
df_cm



Obs.: Nas linhas temos as Classes reais e nas colunas as Classes previstas.

Matriz de Confusão


,0,1
0,5808,351
1,897,1085


In [26]:
# Comparando a Classe Prevista com a Classe Real

df = pd.DataFrame( )
df['Classe_Prevista'] = classe_prevista_teste.ravel()
df['Classe_Real'] = classe_teste
df['Comparacao'] = df['Classe_Prevista'] == df['Classe_Real']

df_comparacao = df.groupby(by='Comparacao').count()

print('\nDe %d classes o modelo ACERTOU %d classes e ERROU %d classes.\n ' % (df_comparacao['Classe_Prevista'].sum(), df_comparacao.iloc[[1], [0]].values, df_comparacao.iloc[[0], [0]].values))

df_comparacao


De 8141 classes o modelo ACERTOU 6893 classes e ERROU 1248 classes.
 


,Classe_Prevista,Classe_Real
Comparacao,,
False,1248,1248
True,6893,6893
